In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from IPython.display import display, HTML

In [ ]:
engine = create_engine(f"mssql+pyodbc://{server}/{dbname}?driver=SQL+Server+Native+Client+11.0")
conn = engine.connect()

In [ ]:
d = pd.read_sql_query("""
SELECT * FROM INFORMATION_SCHEMA.COLUMNS
ORDER BY ORDINAL_POSITION
""", conn).drop("TABLE_CATALOG", 1)

In [ ]:
temp = []
for t in d.TABLE_NAME.unique():
    schema = d.query("TABLE_NAME==@t").TABLE_SCHEMA.iloc[0]
    df = pd.read_sql_query(f"""
    SELECT '{t}' as TABLE_NAME, objname AS COLUMN_NAME, cast([value] AS VARCHAR) as DESCRIPTION
    FROM fn_listextendedproperty (NULL, 'schema', '{schema}', 'table', '{t}', 'column', default); 
    """, conn)
    if df.shape[0] > 0:
        temp.append(df[["TABLE_NAME", "COLUMN_NAME", "DESCRIPTION"]])
temp = pd.concat(temp)
d = d.merge(temp, "left", on=["TABLE_NAME", "COLUMN_NAME"])

In [ ]:
d.loc[:, "DATA_TYPE"] = d.apply(lambda x: f"{x.DATA_TYPE} ({int(x.CHARACTER_MAXIMUM_LENGTH)})" if\
                                (not np.isnan(x.CHARACTER_MAXIMUM_LENGTH)) else x.DATA_TYPE, axis=1)
d.drop("CHARACTER_MAXIMUM_LENGTH", 1, inplace=True)

In [ ]:
c = pd.read_sql_query("""
    select schema_name(t.schema_id) as TABLE_SCHEMA, t.[name] as TABLE_NAME, 
        case when t.[type] = 'U' then 'Table'
            when t.[type] = 'V' then 'View'
            end as [object_type],
        case when c.[type] = 'PK' then 'Primary key'
            when c.[type] = 'UQ' then 'Unique constraint'
            when i.[type] = 1 then 'Unique clustered index'
            when i.type = 2 then 'Unique index'
            end as constraint_type, 
        isnull(c.[name], i.[name]) as constraint_name,
        substring(column_names, 1, len(column_names)-1) as [details]
    from sys.objects t
        left outer join sys.indexes i
            on t.object_id = i.object_id
        left outer join sys.key_constraints c
            on i.object_id = c.parent_object_id 
            and i.index_id = c.unique_index_id
       cross apply (select col.[name] + ', '
                        from sys.index_columns ic
                            inner join sys.columns col
                                on ic.object_id = col.object_id
                                and ic.column_id = col.column_id
                        where ic.object_id = t.object_id
                            and ic.index_id = i.index_id
                                order by col.column_id
                                for xml path ('') ) D (column_names)
    where is_unique = 1
    and t.is_ms_shipped <> 1
    union all 
    select schema_name(fk_tab.schema_id) as TABLE_SCHEMA, fk_tab.name as TABLE_NAME,
        'Table',
        'Foreign key',
        fk.name as fk_constraint_name,
        schema_name(pk_tab.schema_id) + '.' + pk_tab.name
    from sys.foreign_keys fk
        inner join sys.tables fk_tab
            on fk_tab.object_id = fk.parent_object_id
        inner join sys.tables pk_tab
            on pk_tab.object_id = fk.referenced_object_id
        inner join sys.foreign_key_columns fk_cols
            on fk_cols.constraint_object_id = fk.object_id
    union all
    select schema_name(t.schema_id) AS TABLE_SCHEMA, t.[name] AS TABLE_NAME,
        'Table',
        'Check constraint',
        con.[name] as constraint_name,
        con.[definition]
    from sys.check_constraints con
        left outer join sys.objects t
            on con.parent_object_id = t.object_id
        left outer join sys.all_columns col
            on con.parent_column_id = col.column_id
            and con.parent_object_id = col.object_id""", conn)

In [ ]:
temp = ""
for g in d.groupby(["TABLE_SCHEMA", "TABLE_NAME"]):
    temp += f"\r\n<h2>{'.'.join(g[0])}</h2>\r\n"
    temp += "<h3>Schema</h3>\r\n"
    temp += g[1][["COLUMN_NAME", "IS_NULLABLE", "DATA_TYPE", "DESCRIPTION"]]\
    .rename(columns={"COLUMN_NAME": "Attribute", "IS_NULLABLE": "Nullable",
                     "DATA_TYPE": "Data Type", "DESCRIPTION": "Description"})\
    .to_html(col_space=25, na_rep="-", index=False, classes="mystyle")
    g = c.query("TABLE_SCHEMA==@g[0][0] and TABLE_NAME==@g[0][1]")
    if g.shape[0]>0:
        temp += "<h3>Constraints</h3>\r\n"
        temp += g.to_html(col_space=25, na_rep="-", index=False)
display(HTML(temp))


## Sources
https://dataedo.com/kb/query/sql-server/list-all-table-constraints
<br>https://docs.microsoft.com/en-us/sql/relational-databases/system-functions/sys-fn-listextendedproperty-transact-sql